<a href="https://colab.research.google.com/github/xalejandrow/NLP-Project-Tutorial/blob/main/Clase_NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import nltk
import re
import unicodedata

In [11]:
df_raw = pd.read_csv('https://raw.githubusercontent.com/4GeeksAcademy/machine-learning-content/master/assets/spam.csv')

In [3]:
df_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Category  5572 non-null   object
 1   Message   5572 non-null   object
dtypes: object(2)
memory usage: 87.2+ KB


In [12]:
df_raw.sample(10)

,Category,Message
1946,ham,Can ü call me at 10:10 to make sure dat i've w...
2722,ham,I'm working technical support :)voice process.
3676,ham,Whos this am in class:-)
787,ham,It does it on its own. Most of the time it fix...
4281,spam,WINNER!! As a valued network customer you have...
1432,ham,"My painful personal thought- ""I always try to ..."
4812,ham,E admin building there? I might b slightly ear...
4667,ham,Not..tel software name..
1443,ham,"Its ok., i just askd did u knw tht no?"
4415,ham,How stupid to say that i challenge god.You don...


In [13]:
# Tratar de llevar la información de texto a minúsculas
df_raw['Message'] = df_raw['Message'].str.lower()

In [14]:
#df_raw.sample(10)